In [ ]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

In [ ]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [ ]:
#melakukan proses one-hot-encoding.
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [ ]:
#mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [ ]:
#bagi data untuk training dan data untuk testing.
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [ ]:
#ubah setiap kata pada dataset kita ke dalam bilangan numerik 
#dengan fungsi Tokenizer. Setelah tokenisasi selesai, 
#kita perlu membuat mengonversi setiap sampel menjadi sequence.

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [ ]:
# arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16, 
#serta dimensi dari input sebesar nilai num_words pada objek tokenizer.

import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#melatih model kita dengan memanggil fungsi fit().
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 27s - loss: 1.6104 - accuracy: 0.2065 - val_loss: 1.6101 - val_accuracy: 0.1741
Epoch 2/30
26/26 - 24s - loss: 1.6057 - accuracy: 0.2326 - val_loss: 1.6545 - val_accuracy: 0.1741
Epoch 3/30
26/26 - 24s - loss: 1.5655 - accuracy: 0.3197 - val_loss: 1.5935 - val_accuracy: 0.2338
Epoch 4/30
26/26 - 24s - loss: 1.2865 - accuracy: 0.4789 - val_loss: 1.5242 - val_accuracy: 0.2886
Epoch 5/30
26/26 - 24s - loss: 0.8012 - accuracy: 0.6604 - val_loss: 1.9343 - val_accuracy: 0.3731
Epoch 6/30
26/26 - 24s - loss: 0.4583 - accuracy: 0.8396 - val_loss: 2.6211 - val_accuracy: 0.3085
Epoch 7/30
26/26 - 25s - loss: 0.2719 - accuracy: 0.9117 - val_loss: 2.4832 - val_accuracy: 0.3582
Epoch 8/30
26/26 - 25s - loss: 0.1865 - accuracy: 0.9540 - val_loss: 2.8603 - val_accuracy: 0.3284
Epoch 9/30
26/26 - 24s - loss: 0.1308 - accuracy: 0.9614 - val_loss: 2.8574 - val_accuracy: 0.3383
Epoch 10/30
26/26 - 25s - loss: 0.0411 - accuracy: 0.9925 - val_loss: 3.5698 - val_accuracy: 0.3433
Epoch 11/